# FOR Statement

**Applies to:** Databricks SQL, Databricks Runtime 16.3+

Repeat a list of statements **for each row** returned by a query.

```
[label:] FOR [variable_name AS] query DO stmt [...] END FOR [label]
```

Use the optional **variable_name** to qualify column references (e.g. `row.state`). Use **label** with LEAVE or ITERATE. Prefer relational queries when possible; FOR over many rows can be slow.

In [0]:
USE CATALOG healthcare_dev;
USE SCHEMA gold;

## Example: Loop over distinct states from dim_beneficiary (limit rows for demo)

In [0]:
BEGIN
  DECLARE state_list STRING DEFAULT '';
  FOR r AS SELECT DISTINCT sp_state_code AS state FROM dim_cms_beneficiary WHERE sp_state_code IS NOT NULL LIMIT 5 DO
    SET state_list = CONCAT(state_list, COALESCE(r.state, ''), ', ');
  END FOR;
  SELECT state_list AS states_seen;
END

## Example: FOR with LEAVE – sum claim amounts until running total exceeds threshold (using fact_patient_claims)

In [0]:
BEGIN
  DECLARE running DECIMAL(18,2) DEFAULT 0;
  DECLARE threshold DECIMAL(18,2) DEFAULT 50000;
  sumLoop: FOR row AS
    SELECT clm_pmt_amt AS amt FROM fact_cms_outpatient_claims WHERE clm_pmt_amt IS NOT NULL LIMIT 1000
  DO
    SET running = running + COALESCE(row.amt, 0);
    IF running >= threshold THEN
      LEAVE sumLoop;
    END IF;
  END FOR sumLoop;
  SELECT running AS total_until_threshold, threshold;
END